
# Advance Lane Finding Project

## Chessboard Calibration

In [ ]:
import cv2
import glob
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

imgpts = []

chessColumns = 9
chessRows = 6

chessDim = (chessColumns, chessRows)

# List comprehension instead of glob to make it easier to pick out
# a particular image to undistort.
calibimgpaths = ['camera_cal/calibration{}.jpg'.format(i) for i in range(1,20+1)]
print(calibimgpaths)

calibimgs = list(map(cv2.imread, calibimgpaths))

for img in calibimgs:
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, chessDim)
    #print('ret: {}, corners: {}'.format(ret, corners))
    if ret:
        imgpts.append(corners)
    
    imgcorners = img.copy()
    #cv2.drawChessboardCorners(imgcorners, chessDim, corners, ret)
    #plt.imshow(img)

# 3D points for chessboard lying in xz-plane
objp = np.zeros((chessColumns*chessRows,3), np.float32)
objp[:,:2] = np.mgrid[0:chessColumns,0:chessRows].T.reshape(-1,2) # keep third dimension 

objpts = [objp] * len(imgpts)

imageDim = calibimgs[0].shape[1::-1]

ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, imageDim, None, None)

## Undistortion

In [ ]:
# Was a bit tricky to work with cv2.undistortPoints() since it doesn't output image coordinates
# when I called it like this: cv2.undistort(distorted, cameraMatrix, distCoeffs)
# Also doing this calculation once should be faster than letting cv2.undistort do it every time.
newCameraMatrix,roi = cv2.getOptimalNewCameraMatrix(cameraMatrix,distCoeffs,imageDim,1.,imageDim)
mapx,mapy = cv2.initUndistortRectifyMap(cameraMatrix,distCoeffs,None,newCameraMatrix,imageDim,cv2.CV_32FC1)
def undistort_road_img(img):
    return cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

print('map x and y shapes: {} {}'.format(mapx.shape, mapy.shape))
def undistort_road_img_points(pts):
    #res = [(mapx[pt[1],pt[0]], mapy[pt[1],pt[0]]) for pt in np.rint(pts).astype(int)]
    #return np.array(res, np.float32)
    pts = pts[:,np.newaxis,:]
    res = cv2.undistortPoints(pts, cameraMatrix, distCoeffs, P=newCameraMatrix)
    res = np.squeeze(res)
    return res

# Show one img restored
distorted = calibimgs[3-1]
undistorted = undistort_road_img(distorted) #cv2.undistort(distorted, cameraMatrix, distCoeffs)

plt.subplot(121)
plt.title('Distorted chessboard')
plt.imshow(distorted)

plt.subplot(122)
plt.title('Undistorted chessboard')
plt.imshow(undistorted)

## Thresholding

In [ ]:
# Thresholding of the warped image.
# Code from udacity exercise "Color and Gradient"
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    print(s_thresh)
    print(sx_thresh)
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    #print('s_channel: {}'.format(s_channel[-100:-80,-100:-80]))
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    print(sobelx.shape)
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    print(scaled_sobel.shape)
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    print('sxbinary shape: {}'.format(sxbinary.shape))
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255
    print('sxbinary shape: {}'.format(sxbinary.shape))
    print('num sobel in threshold: {}'.format(sum(sum(sxbinary))))
    
    kernel = np.ones((7,7),np.uint8)
    sxmorph = cv2.dilate(sxbinary,kernel,iterations = 1)
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255
    print('num s in theshold: {}'.format(sum(sum(s_binary))))
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    #return np.logical_and(sxmorph == 1, s_binary == 1)
    print((sxbinary == 1).dtype)
    print('scaled_sobel shape: {}'.format(scaled_sobel.shape))
    result = np.zeros_like(scaled_sobel)
    print('result shape: {}'.format(result.shape))
    result[(sxbinary > 0) | (s_binary > 0)] = 255
    return result

distorted = cv2.imread('test_images/test5.jpg')
undistorted = undistort_road_img(distorted)

thresholded = pipeline(undistorted, s_thresh=(160,255), sx_thresh=(20, 100))
cv2.imwrite('output_images/thresholded_image.jpg', thresholded)

plt.subplot(121)
plt.title('Undistorted road')
plt.imshow(cv2.cvtColor(undistorted, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title('Thresholded road')
plt.imshow(thresholded, cmap='gray')

## Unwarped view of lane area

In [ ]:
# Distorted coordinates for straight_lines2.jpg
#srcpts = np.array([(308,656), (615,435), (666,435), (1012,656)], np.float32)
srcpts = np.array([(308,656), (588,455), (697,455), (1012,656)], np.float32)
srcptsUndistorted = undistort_road_img_points(srcpts)

# From Section 35. Measuring Curvature in course "We could do this in detail by measuring out the
# physical lane in the field of view of the camera, but for this project, you can assume that if
# you're projecting a section of lane similar to the images above, the lane is about 30 meters
# long and 3.7 meters wide."
unwrappedDim = (1280,720)
# The spacing in pixels to the side of the lane
sidemargin = 200
#dstpts = np.multiply(np.array([(0,1), (0,0), (1,0), (1,1)], np.float32), np.float32(unwrappedDim))
dstpts = np.array([
    (sidemargin,unwrappedDim[1]),
    (sidemargin,0),
    (unwrappedDim[0]-sidemargin,0),
    (unwrappedDim[0]-sidemargin,unwrappedDim[1])],
    np.float32)
print(dstpts)
M = cv2.getPerspectiveTransform(srcptsUndistorted, dstpts)
Minv = np.linalg.inv(M)

def unwarp_lane(img):
    warped = cv2.warpPerspective(img, M, tuple(unwrappedDim), flags=cv2.INTER_LINEAR)
    return warped

def from_unwarped_to_warped(warped):
    #return cv2.warpPerspective(warped, M, tuple(imageDim), cv2.INTER_LINEAR | cv2.WARP_INVERSE_MAP)
    return cv2.warpPerspective(warped, Minv, tuple(imageDim), cv2.INTER_LINEAR)

kernel = np.ones((3,3),np.uint8)
# The lane contours have pixels set, but the middle doesn't a closing homography
# fixes that and makes the sliding window search better.
def close_binary_lane_img(thresholded):
    return cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

thresholded = close_binary_lane_img(thresholded)

warpedthresholded = unwarp_lane(thresholded)

undistortedWithSrcPoints = undistorted.copy()
thresholdedWithSrcPoints = np.array(cv2.merge((thresholded,thresholded,thresholded)),np.uint8)
for img in [undistortedWithSrcPoints, thresholdedWithSrcPoints]:
    for pt in srcptsUndistorted:
        cv2.circle(img, (pt[0],pt[1]), 6, (0,0,255), 2)

plt.subplot(131)
plt.title('Undistorted road')
plt.imshow(cv2.cvtColor(undistortedWithSrcPoints, cv2.COLOR_BGR2RGB))
cv2.imwrite('output_images/undistorted_with_warppoints.jpg', undistortedWithSrcPoints)

plt.subplot(132)
plt.title('Thresholded')
plt.imshow(cv2.cvtColor(thresholdedWithSrcPoints, cv2.COLOR_BGR2RGB))
    
plt.subplot(133)
plt.title('Warped and thresholded')
plt.imshow(warpedthresholded, cmap='gray')
cv2.imwrite('output_images/unwarpedlines.jpg', warpedthresholded)

## Finding of lane lines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2

# Code heavily adopted from section "Sliding Window Search"

# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
        
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids

def draw_window_centroids(window_centroids, warped):
    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 0.7, template, 0.5, 0.0) # overlay the orignal road image with window results
        #output = warpedthresholded
        #print('warpage.shape: {}'.format(warpage.shape))
        #smallwarpage = warpage[700:719,180:210,:]
        #print('smallwarpage: {}'.format(smallwarpage))
        #print('histogram: {}'.format(np.histogram(warpage.flatten().reshape(-1,3))))
        #print('row: {}'.format(warpage[719,:,:]))
        #print('row histogram: {}'.format(cv2.calcHist(warpage[719,:,:], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])))
        # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    return output

window_centroids = find_window_centroids(warpedthresholded, window_width, window_height, margin)

output = draw_window_centroids(window_centroids, warpedthresholded)

cv2.imwrite('output_images/lane_location.jpg', output)

# Display the final results
plt.imshow(output)
plt.title('window fitting results')
plt.show()


## Fit polygon to the points

In [ ]:
# Heavily adopted from section "Finding the lines"
# part "Implement Sliding Windows and Fit a Polynomial".
def fit_poly_around_centroids(window_centroids, img):
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Set the width of the windows +/- margin
    margin = 100

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    for level in range(0,len(window_centroids)):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (level+1)*window_height
        win_y_high = img.shape[0] - level*window_height
        leftx_current, rightx_current = window_centroids[level]
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit,right_fit

left_fit,right_fit = fit_poly_around_centroids(window_centroids, warpedthresholded)

## Visualization of the fitted lines

In [ ]:
# Heavily adopted from section "Finding the lines"
# part "Visualization".
def draw_fitted_polylines(img):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    polygon_img = cv2.merge((img,img,img))
    
    def draw_helper(xvalues, yvalues):
        pts = cv2.merge((xvalues,yvalues))
        pts = np.array(pts, int)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(polygon_img,[pts],False,(0,255,255), 4)

    draw_helper(left_fitx,ploty)
    draw_helper(right_fitx,ploty)
    return polygon_img

polygon_img = draw_fitted_polylines(warpedthresholded)
plt.imshow(polygon_img)

In [ ]:
# Modified version of code from section "Tips and Tricks for the Project"
def draw_lane_on_image(warpedthresholded, undistorted, left_fit, right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lines on
    color_warp = np.zeros((warpedthresholded.shape[0], warpedthresholded.shape[1], 3), dtype=np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = from_unwarped_to_warped(color_warp)

    # Combine the result with the original image
    result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
    return result

result = draw_lane_on_image(warpedthresholded, undistorted, left_fit, right_fit)
plt.imshow(result)

## Video pipeline

In [ ]:
from moviepy.editor import VideoFileClip

def process_frame(frame):
    undistorted = undistort_road_img(frame)

    thresholded = pipeline(undistorted, s_thresh=(160,255), sx_thresh=(20, 100))

    thresholded = close_binary_lane_img(thresholded)

    warpedthresholded = unwarp_lane(thresholded)
    
    window_centroids = find_window_centroids(warpedthresholded, window_width, window_height, margin)

    left_fit,right_fit = fit_poly_around_centroids(window_centroids, warpedthresholded)
    
    image_with_marked_lane = draw_lane_on_image(warpedthresholded, undistorted, left_fit, right_fit)

    return image_with_marked_lane

def process_road_video2(infilename, outfilename):
    cap = None
    videoOut = None
    try:
        cap = skvideo.io.VideoCapture(infilename)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        videoOut = cv2.VideoWriter(outfilename,fourcc, 20.0, (640,480))
        while(True):
            ret, frame = cap.read()
            if not ret:
                print(ret)
                break
                
            outframe = process_frame(frame)
            plt.imshow(outframe)
            out.write(outframe)
            break
    finally:
        for io in [cap, videoOut]:
            if io: io.release()

def process_road_video(infilename, outfilename):
    clip = VideoFileClip(infilename)
    clip_blurred = clip.fl_image(process_frame)
    clip_blurred.write_videofile(outfilename)
                
process_road_video('project_video.mp4', 'output_images/processed_project_video.mp4')